**Nandini Bajaj 18CY20020**

Lab Assignment 5


In [16]:
import pandas as pd
import numpy as np
import math
import random
import matplotlib.pyplot as plt

In [17]:
step_1 = pd.read_csv("step_1.csv")
step_2 = pd.read_csv("step_2.csv")
step_3 = pd.read_csv("step_3.csv")
step_4 = pd.read_csv("step_4.csv")
step_5 = pd.read_csv("step_5.csv")

In [18]:
l_steps = [step_1, step_2, step_3, step_4, step_5]

In [19]:
#12 - 6 lennard jones
#r,rc,eps,sig
def lennard_jones(r, rc, eps, sig, m = 12, n = 6):
    if r>=rc:     
        return 0
    a1 = sig/r
    a = 4*eps*(pow(a1,m)-pow(a1,n))
    return a

In [20]:
#:L=26
def minimum_image_convention(dis, L):
    if dis > L/2:
        dis = dis - L
    elif dis <= -L/2:
        dis = dis + L
    return dis

In [21]:
def cal_pot_energy(pos, N, L, rc, eps, sig):
    #potential energy
    v_net = 0.0    
    
    for i in range(N-1):
        x1 = pos.iloc[i][0]
        y1 = pos.iloc[i][1]
        z1 = pos.iloc[i][2]
        for j in range(i+1, N):
            x2 = pos.iloc[j][0]
            y2 = pos.iloc[j][1]
            z2 = pos.iloc[j][2]
            
            #minimum image convention
            dx = x2 - x1
            dy = y2 - y1
            dz = z2 - z1
            
            dx = minimum_image_convention(dx, L)
            dy = minimum_image_convention(dy, L)
            dz = minimum_image_convention(dz, L)
            
            r_ij = math.sqrt(dx*dx + dy*dy + dz*dz)
            v_net += lennard_jones(r_ij, rc, eps, sig)
    return v_net

In [22]:
#L
def pbc_on_one_particle(x, y, z, L):
        applied_pbc = False
        if x>L/2: 
            x = x - L
            applied_pbc = True
        elif x<= -L/2:
            x = x + L
            applied_pbc = True
        if y > L/2: 
            y = y - L
            applied_pbc = True
        elif y<= -L/2: 
            y = y + L
            applied_pbc = True
        if z > L/2: 
            z = z - L
            applied_pbc = True
        elif z<= -L/2: 
            z = z + L
            applied_pbc = True
        
        return x,y,z, applied_pbc
        

In [23]:
def pbc(df, n, L):
    #total no. of pbc in 1 iter
    ans = 0      
    for i in range(n):
        x = df.iloc[i][0]
        y = df.iloc[i][1]
        z = df.iloc[i][2]

        x,y,z,applied_pbc = pbc_on_one_particle(x,y,z, L)
        df.iloc[i][0] = x
        df.iloc[i][1] = y
        df.iloc[i][2] = z
        if(applied_pbc == True):
            ans += 1
    
    return df, ans

In [24]:
def distance_in_3D(ith, jth):
    x1 = ith[0]
    y1 = ith[1]
    z1 = ith[2]
    x2 = jth[0]
    y2 = jth[1]
    z2 = jth[2]
    return math.sqrt((x1-x2)**2 + (y1-y2)**2 + (z1-z2)**2)

In [25]:
def force_cal(pos, N, eps, sig):
    l_a = [0.0,0.0,0.0]
    arr = [l_a for i in range(N)]
    force = pd.DataFrame(arr, columns = ["x", "y", "z"])
    
    #calculate force for every particle with every other particle
    for i in range(N):
        x1 = pos.iloc[i][0]
        y1 = pos.iloc[i][1]
        z1 = pos.iloc[i][2]
        for j in range(i+1,N):
            r_ij = distance_in_3D(list(pos.iloc[i]), list(pos.iloc[j]))
            x2 = pos.iloc[j][0]
            y2 = pos.iloc[j][1]
            z2 = pos.iloc[j][2]
            temp = 24*eps*((2*pow(sig,12))/pow(r_ij,14) - pow(sig, 6)/pow(r_ij, 8)) 
            force.iloc[i][0] += temp*(x1-x2)
            force.iloc[j][0] += temp*(x2-x1)
            force.iloc[i][1] += temp*(y1-y2)
            force.iloc[j][1] += temp*(y2-y1)
            force.iloc[i][2] += temp*(z1-z2)
            force.iloc[j][2] += temp*(z2-z1)
    #mass is 1 so force is same as acceleration
    return force

In [26]:
sig=float(input("Enter the value of sigma:"))
eps=float(input("Enter the value of epsilon:"))
L=float(input("Enter the cell dimension:"))
rc=float(input("Enter the cut-off radius:"))

Enter the value of sigma:3.4
Enter the value of epsilon:1
Enter the cell dimension:26
Enter the cut-off radius:10


In [27]:
for a, i in enumerate(l_steps):
    pos = i
    v_tot = cal_pot_energy(pos, pos.shape[0], L, rc, eps, sig)
    print("Potential Energy for step ", a+1, " = ", v_tot, " kJ / mol")

Potential Energy for step  1  =  -2067.9064297969207  kJ / mol
Potential Energy for step  2  =  -2047.0301909069062  kJ / mol
Potential Energy for step  3  =  -2056.215577890881  kJ / mol
Potential Energy for step  4  =  -2055.5009115162293  kJ / mol
Potential Energy for step  5  =  -2041.4760724951134  kJ / mol


In [28]:
force_step_1 = force_cal(step_1, step_1.shape[0], eps, sig)
force_step_1

,x,y,z
0,-1.911426,0.511834,0.311538
1,2.154337,2.861268,-6.465524
2,-0.696979,1.475604,-0.500409
3,-3.368000,2.979613,3.102720
4,0.593019,2.310170,-3.254401
...,...,...,...
360,-2.454897,-4.199239,-15.078637
361,-2.354922,3.856777,1.315494
362,0.746483,-0.048458,-1.322170
363,2.176271,0.023178,8.137600


In [29]:
force_step_1.to_csv("force_step1_calculated.csv")